# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
cities = pd.read_csv("output_data/cities.csv", encoding="utf-8")
cities.head()

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
0,933778,Francistown,BW,-21.1700,27.5079,66.52,70,100,14.74
1,2075265,Busselton,AU,-33.6500,115.3333,61.05,66,100,9.28
2,2618795,Klaksvík,FO,62.2266,-6.5890,52.72,85,100,21.00
3,3366880,Hermanus,ZA,-34.4187,19.2345,56.89,75,0,7.56
4,371745,Kutum,SD,14.2000,24.6667,73.09,24,59,7.09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
humidity = cities["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities[["Lat", "Lng"]]

In [12]:
fig = gmaps.figure()
heat_layer=gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
narrow_city_df = cities.loc[(cities["Temperature"] > 70) & (cities["Temperature"] < 80) & (cities["Clouds"] == 0), :]
narrow_city_df = narrow_city_df.dropna(how='any')
narrow_city_df.reset_index(inplace=True)
del narrow_city_df['index']
narrow_city_df.head()

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
0,3433899,Formosa,AR,-26.1775,-58.1781,71.58,68,0,12.66
1,2141305,Fayaoué,NC,-20.6500,166.5333,73.35,64,0,14.56
2,347796,Sohag,EG,26.5570,31.6948,70.23,43,0,8.16
3,108410,Riyadh,SA,24.6877,46.7219,75.34,38,0,3.74
4,2063042,Port Hedland,AU,-20.3167,118.5667,78.85,69,0,6.91


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
hotellist = []

for i in range(len(narrow_city_df)):
    lat = narrow_city_df.loc[i]['Lat']
    lng = narrow_city_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
narrow_city_df["Hotel Name"] = hotellist
narrow_city_df = narrow_city_df.dropna(how='any')
narrow_city_df.head()

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed,Hotel Name
0,3433899,Formosa,AR,-26.1775,-58.1781,71.58,68,0,12.66,Formosa
1,2141305,Fayaoué,NC,-20.6500,166.5333,73.35,64,0,14.56,Fayaoue
2,347796,Sohag,EG,26.5570,31.6948,70.23,43,0,8.16,Sohag
3,108410,Riyadh,SA,24.6877,46.7219,75.34,38,0,3.74,Riyadh
4,2063042,Port Hedland,AU,-20.3167,118.5667,78.85,69,0,6.91,Port Hedland


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrow_city_df.iterrows()]
locations = narrow_city_df[["Lat", "Lng"]]

In [24]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))